In [36]:
from math import log
from sklearn import datasets
import numpy as np
import pandas as pd

def entropy(column):
    probabilities=[]
    for x in column.value_counts():
        probabilities.append(x/sum(col1.value_counts()))
    return -sum([p*log(p,2) for p in probabilities])

In [335]:
iris=datasets.load_iris()
iris_data=iris['data']
iris_target=iris['target']
data=[]
for index,row in enumerate(iris_data):
    new_row=np.append(row,[iris_target[index]])
    data.append(new_row.tolist())
    
# data #list of lists
columns=iris.feature_names
columns.append('species')
df=pd.DataFrame(data,columns=columns)
df.describe()
# df.dtypes
df.tail(80)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
70,5.9,3.2,4.8,1.8,1.0
71,6.1,2.8,4.0,1.3,1.0
72,6.3,2.5,4.9,1.5,1.0
73,6.1,2.8,4.7,1.2,1.0
74,6.4,2.9,4.3,1.3,1.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [195]:
def importance(attributes,examples):
    label_column=examples.iloc[:,-1:]
    current_entropy=entropy(label_column)
    max_gain=0.0
    important_attribute=attributes[0]
    for a in attributes:
        col=examples[a]
        remainder=0.0
        for v in col.unique():
            subset=examples[examples[a]==v]
            subset_entropy=entropy(subset.iloc[:,-1:])
            subset_probability=len(subset)/len(examples)
            remainder=remainder+(subset_entropy*subset_probability)
        gain=current_entropy-remainder
        if gain>max_gain:
            max_gain=gain
            important_attribute=a
    return important_attribute
A=importance(df.columns.drop('species'),df)
A

'petal length (cm)'

In [272]:
class Node:
    def __init__(self):
        self.children = []
        self.branch_value = None
        self.attribute = None
        self.leaf=None
        
        
    def __str__(self):
        return """
            leaf:{0},
            attribute:{1},
            branch_value:=={2}
        """.format(self.leaf,self.attribute,self.branch_value)

In [331]:
def learn_decision_tree(examples,attributes,parent_examples,branch_value=None,parent=None):
    """
        Expects:
            - examples: The branch examples - Pandas Dataframe including the label. Label should be the last column
            - attributes: List/Pandas Index of the attributes excluding the label
            - parent_examples: The Parent Node examples - Pandas Dataframe including the label. Label should be the last column
    """
    if examples.empty:
#         print("Examples exhausted")
        node=Node()
        node.branch_value=branch_value
        node.leaf=max(parent_examples.iloc[:,-1:]).index
        return node
    elif len(examples.iloc[:,-1].unique())==0:
#         print("Reached Leaf with Uniform Class")
        node=Node()
        node.branch_value=branch_value
        node.leaf=examples.iloc[:,-1:][0]
        return node
    elif len(attributes)==0:
#         print("Attributes exhausted")
        node=Node()
        node.branch_value=branch_value
        node.leaf=examples[examples.columns[-1]].mode().tolist()
        return node
    else:
#         print("Got Attributes :",str(attributes))
#         print("Got examples :",str(examples))
        A=importance(attributes,examples)
        
#         print("Got ATTRIBUTE :"+str(A))
        node=Node()
        node.attribute=A
        node.branch_value=branch_value
#         print("\n --- \n")
        
        for v in examples[A].unique():
            subset=examples[examples[A]==v]
            new_attributes_set=attributes.drop(A)
            subtree=learn_decision_tree(subset,new_attributes_set,examples,branch_value=v)
            node.children.append(subtree)
        
        return node
    

attributes=df.columns.drop('species')
tree=learn_decision_tree(df,attributes,df)

In [332]:
print(tree)
for c in tree.children:
    print(c)
    for a in c.children:
        print(a)
        for d in a.children:
            print(d)
            for e in d.children:
                print(e)
                break
            break
        break
    break


            leaf:None,
            attribute:petal length (cm),
            branch_value:None,
        

            leaf:None,
            attribute:sepal length (cm),
            branch_value:1.4,
        

            leaf:None,
            attribute:petal width (cm),
            branch_value:5.1,
        

            leaf:None,
            attribute:sepal width (cm),
            branch_value:0.2,
        

            leaf:[0.0],
            attribute:None,
            branch_value:3.5,
        


In [336]:
def predict_from_tree(instance,root):
    if root.leaf == None:
        test=root.attribute
        for child in root.children:
            if instance[test][0]==child.branch_value:
                return predict_from_tree(instance,child)
    else:
        return root.leaf
test_data=pd.DataFrame([[5.9,3.2,4.8,1.8]],columns=df.columns.drop('species'))
prediction=predict_from_tree(test_data,tree)
test_data.head()
prediction

[1.0]

'species'